In [ ]:
!pip install fuzzywuzzy

In [ ]:
#collaborative filtering

In [ ]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
movies2 = pd.read_csv('/content/movies.csv', usecols=['movieId', 'title'], dtype={'movieId':'int32', 'title':'str'})
ratings2 = pd.read_csv('/content/ratings.csv', usecols=['userId', 'movieId', 'rating'], dtype={'userId':'int32', 'movieId':'int32', 'rating':'float64'}).fillna(0)
movies2_users = ratings2.pivot(index='movieId', columns='userId', values='rating').fillna(0)
movies2_matrix = csr_matrix(movies2_users.values)

In [ ]:
knn = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=20)
knn.fit(movies2_matrix)

NearestNeighbors(metric='cosine', n_neighbors=20)

In [ ]:
def recommender1(matrix, user_input, movies2, model):
  movie_index = process.extractOne(user_input, movies2['title'])[2]
  dist, idc = model.kneighbors(matrix[movie_index], n_neighbors=6)
  idc = idc[0][idc[0] != movie_index]
  for i in idc:
    print(movies2['title'][i])

In [ ]:
#content based filtering

In [ ]:
movies = pd.read_csv('/content/movies.csv')
ratings = pd.read_csv('/content/ratings.csv')
movies['year'] = movies.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies['year'] = movies.year.str.extract('(\d\d\d\d)', expand=False)
movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '', regex=True)
movies['title'] = movies['title'].apply(lambda x: x.strip())
movies['title'] = movies.title.str.lower()
movies['genres'] = movies.genres.str.split('|')

In [ ]:
for index, row in movies.iterrows():
    for genre in row['genres']:
        movies.at[index, genre] = 1
movies = movies.fillna(0)
#rating dataset
ratings = ratings.drop('timestamp', axis=1)

In [ ]:
user_input = []
while True:
  movie_title = input("Enter movie title (or 'q' to quit): ").lower()
  if movie_title == 'q':
    break
  rating = float(input("Enter rating (1-5): "))
  user_input.append({'title': movie_title, 'rating': rating})
input_movies = pd.DataFrame(user_input)
user_input2 = input_movies.iloc[0]['title']

Enter movie title (or 'q' to quit): drive
Enter rating (1-5): 5
Enter movie title (or 'q' to quit): toy story
Enter rating (1-5): 4.5
Enter movie title (or 'q' to quit): jumanji
Enter rating (1-5): 2
Enter movie title (or 'q' to quit): q


In [ ]:
input_id = movies[movies['title'].isin(input_movies['title'].to_list())]
movie_input = pd.merge(input_id, input_movies)
movie_input = movie_input.drop('genres', axis=1).drop('year', axis=1)
user_movies  = movies[movies['movieId'].isin(movie_input['movieId'].tolist())]
user_movies = user_movies.reset_index(drop=True)
user_genres = user_movies.drop('movieId', axis=1).drop('title', axis=1).drop('genres', axis=1).drop('year', axis=1)
user_profile = user_genres.transpose().dot(movie_input['rating'])
genre_df = movies.set_index(movies['movieId'])
genre_df = genre_df.drop('movieId', axis=1).drop('title', axis=1).drop('genres', axis=1).drop('year', axis=1)

In [ ]:
def recommender2(genres, user, movies):
  rec_movies = ((genres*user).sum(axis=1))/(user.sum())
  rec_movies = rec_movies.sort_values(ascending=False)
  recommendations = movies.loc[movies['movieId'].isin(rec_movies.head().keys())]['title'].tolist()
  for movie_name in recommendations:
    print(movie_name)

In [ ]:
print("Top recommendations.....")
recommender2(genre_df, user_profile, movies)
print("What other users liked....")
recommender1(movies2_matrix, user_input2, movies2, knn)

Top recommendations.....
watership down
who framed roger rabbit?
motorama
wonderful world of the brothers grimm, the
rubber
What other users liked....
Star Trek: Generations (1994)
Hot Lead and Cold Feet (1978)
Unforgiven (1992)
Junior (1994)
